# input

---

In [116]:
# Google Colab

In [117]:
# from google.colab import files
# import io

In [118]:
# uploaded = files.upload()
# df = pd.read_csv(io.BytesIO(uploaded['movies_metadata.csv']))

---

In [119]:
# Local

In [120]:
import pandas as pd
import numpy as np
from scipy import stats
from statistics import median

In [121]:
df = pd.read_csv('./movielens/movies_metadata.csv', low_memory=False)

# Limpieza

In [122]:
# Columnas seleccionadas
df = df[['imdb_id', 'title', 'budget', 'revenue', 'genres', 'original_language', 'overview',
         'release_date', 'runtime', 'popularity', 'production_companies',
         'production_countries', 'belongs_to_collection',  'vote_average', 'vote_count']]

In [123]:
## REVEER LIMITE INFERIOR!
min_revenue = 1000
df = df[df.revenue >= min_revenue]

In [124]:
## REVEER LIMITE INFERIOR!
min_budget = 1000
# budget lo lee como object ---> 
df['budget'] = df['budget'].apply(lambda x: float(x) if x.isnumeric() else 0.)
df = df[df.budget >= min_budget]

In [125]:
df['original_language'] = df.original_language.apply(
    lambda x: x if isinstance(x, str) and x.isalpha() else '')

In [126]:
#Parse date
df['release_date'] = pd.to_datetime(df['release_date'], errors='coerce')
df['release_date_year'] = df['release_date'].dt.year

In [127]:
#Me quedo con las peliculas del 1985 en adelante
#df = df[df.release_date_year > 1985]

In [128]:
#Bool si es de una coleccion
df['belongs_to_collection'] = df['belongs_to_collection'].apply(lambda x:True if isinstance(x, str) else False)

In [130]:
# Lista de generos
df['genres'] = df.genres.apply(lambda x: [i['name']  for i in eval(x)])

In [131]:
# Lista de production countries
df['production_countries'] = df.production_countries.apply(lambda x: [i['name']  for i in eval(x)])

In [132]:
# Lista de production companies
df['production_companies'] = df.production_companies.apply(lambda x: [i['name']  for i in eval(x)])

In [133]:
# Reveer runtime margins
upper_lim = np.mean(df.runtime) + np.std(df.runtime)*2
lower_lim = np.mean(df.runtime) - np.std(df.runtime)*2
df = df[(df.runtime > lower_lim) & (df.runtime < upper_lim)]

In [134]:
upper_lim, lower_lim

(153.92775063859037, 65.94760240226822)

In [135]:
# Drop na?
df.dropna(subset=['overview', 'runtime', 'release_date'], inplace=True)

Quito outliers usando la median absolute deviation porque no tiene una distribución de cola pesada.
Hago la mediana +- 2.5 MAD

In [136]:
mad = stats.median_abs_deviation(df.revenue)
med = median(df.revenue)

upp_lim = med + 2.5*mad
low_lim = med - 2.5*mad

df = df[(df.revenue > low_lim) & (df.revenue < upp_lim)]

Creo dos columnas. Una divido al revenue en 4 cuartiles y la otra es dividir al revenue en 10 deciles

In [137]:
labels_4 = ['Bajo', 'Moderado', 'Alto', 'Muy Alto']
labels_10 = ['Bajisimo','Muy Bajo','Bajo','Moderado Bajo','Moderado Medio','Moderado Alto','Alto Bajo','Alto Medio','Muy Alto','Altisimo']

df['quantile_4'] = pd.qcut(df['revenue'], q=4, precision = 0, labels = labels_4)
df['quantile_10'] = pd.qcut(df['revenue'], q=10, precision=0, labels = labels_10)


In [138]:
#CREO TABLAS PARA SABER QUE THRESHOLD CORRESPONDE A CADA TIER
results4, bin_edges4 = pd.qcut(df['revenue'],
                            q=4,
                            labels=labels_4,
                            retbins=True)

results_table_4 = pd.DataFrame(zip(bin_edges4, labels_4),
                            columns=['Threshold', 'Tier'])


results10, bin_edges10 = pd.qcut(df['revenue'],
                            q=10,
                            labels=labels_10,
                            retbins=True)

results_table_10 = pd.DataFrame(zip(bin_edges10, labels_10),
                            columns=['Threshold', 'Tier'])

Creo otras dos columnas que armen 4 y 10 rangos del mismo tamaño en terminos de revenue en cada uno

In [139]:
df['4cuts'] = pd.cut(df['revenue'], bins = 4, labels = labels_4)
df['10cuts'] = pd.cut(df['revenue'], bins = 10, labels = labels_10)

## Dummies

In [140]:
from sklearn.preprocessing import MultiLabelBinarizer

In [141]:
mlb = MultiLabelBinarizer()

In [142]:
genres_one_h = mlb.fit_transform(df.genres)

In [143]:
df.reset_index(inplace=True)

In [144]:
genres_oneh.shape

(5028, 20)

In [145]:
mlb.classes_

array(['Action', 'Adventure', 'Animation', 'Comedy', 'Crime',
       'Documentary', 'Drama', 'Family', 'Fantasy', 'Foreign', 'History',
       'Horror', 'Music', 'Mystery', 'Romance', 'Science Fiction',
       'TV Movie', 'Thriller', 'War', 'Western'], dtype=object)

In [146]:
df = pd.concat([df, pd.DataFrame(columns=mlb.classes_, data=genres_one_h)], axis=1)